In [ ]:
!pip install pypandoc
!pip install --upgrade protobuf tf2onnx keras2onnx onnxruntime tensorflow keras numpy azureml-sdk[automl]

In [ ]:
SUBSCRIPTION_ID='<Add your subs. id>'
WORKSPACE='<Add your workspace name>'

In [ ]:
#Access Azure workspace
from azureml.core import Workspace
try:
    ws = Workspace(subscription_id = SUBSCRIPTION_ID, resource_group = WORKSPACE, workspace_name = WORKSPACE)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace")

In [ ]:
##import libraries
import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

#Import dataset and set it up to train and test sets
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


In [ ]:
#simplify images /255
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
#build model_v1
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
#Complie the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#train the model
model.fit(train_images, train_labels, epochs=5)

In [ ]:
#test the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
image = test_images[99]
plt.imshow(image)
x = (np.expand_dims(image, 0))

predictions = model.predict(x)
print(predictions)

model.save('model.h5')

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from keras.models import load_model

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('model.h5')
    model = load_model(model_path)

def run(raw_data):
    data = np.asarray(json.loads(raw_data)['data'])
    return model.predict(data)

In [ ]:
%%writefile myenv.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.1

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- keras


In [ ]:
%%time
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "MNIST",  "method" : "keras"}, 
                                               description='Predict MNIST with keras')

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='mnist',
                                       deployment_config=aciconfig,
                                       models=['model.h5'],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

print(service.scoring_uri)


In [ ]:
import json

print(x.tolist(), np.asarray(x.tolist()))
test_sample = json.dumps({'data': x.tolist()})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = service.run(input_data = test_sample)
print(prediction)